# ANALISI E REVISIONE DEL PROGETTO LARSPLITTING 2D 

## CLASSE REFACTORING: pointInPolygonClassification

Variabili utili per testare il funzionamento


In [29]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads


In [30]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);    #vertici del modello 2D
V3 = hcat([[0.,0,0],[1,0,3],[1,1,2],[0,1,1],[2,1,0]]...);   #vertici del modello 3D
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];             #spigoli del modello
bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];  #bounding box
dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])  #dizionario intervallo/indice
cov = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]    #intersezioni tra bounding box

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

## Versione iniziale di pointInPolygonClassification

In [31]:
function pointInPolygonClassification(V,EV)
    function pointInPolygonClassification0(pnt)
        x,y = pnt
        xmin,xmax,ymin,ymax = x,x,y,y
        tilecode = setTile([ymax,ymin,xmax,xmin])
        count,status = 0,0

        for (k,edge) in enumerate(EV)
            p1,p2 = V[:,edge[1]],V[:,edge[2]]
            (x1,y1),(x2,y2) = p1,p2
            c1,c2 = tilecode(p1),tilecode(p2)
            c_edge, c_un, c_int = c1⊻c2, c1|c2, c1&c2

            if (c_edge == 0) & (c_un == 0) return "p_on"
            elseif (c_edge == 12) & (c_un == c_edge) return "p_on"
            elseif c_edge == 3
                if c_int == 0 return "p_on"
                elseif c_int == 4 count += 1 end
            elseif c_edge == 15
                x_int = ((y-y2)*(x1-x2)/(y1-y2))+x2
                if x_int > x count += 1
                elseif x_int == x return "p_on" end
            elseif (c_edge == 13) & ((c1==4) | (c2==4))
                    crossingTest(1,2,status,count)
            elseif (c_edge == 14) & ((c1==4) | (c2==4))
                    crossingTest(2,1,status,count)
            elseif c_edge == 7 count += 1
            elseif c_edge == 11 count = count
            elseif c_edge == 1
                if c_int == 0 return "p_on"
                elseif c_int == 4 crossingTest(1,2,status,count) end
            elseif c_edge == 2
                if c_int == 0 return "p_on"
                elseif c_int == 4 crossingTest(2,1,status,count) end
            elseif (c_edge == 4) & (c_un == c_edge) return "p_on"
            elseif (c_edge == 8) & (c_un == c_edge) return "p_on"
            elseif c_edge == 5
                if (c1==0) | (c2==0) return "p_on"
                else crossingTest(1,2,status,count) end
            elseif c_edge == 6
                if (c1==0) | (c2==0) return "p_on"
                else crossingTest(2,1,status,count) end
            elseif (c_edge == 9) & ((c1==0) | (c2==0)) return "p_on"
            elseif (c_edge == 10) & ((c1==0) | (c2==0)) return "p_on"
            end
        end
        if (round(count)%2)==1
        	return "p_in"
        else
        	return "p_out"
        end
    end
    return pointInPolygonClassification0
end

pointInPolygonClassification (generic function with 1 method)

In [47]:
@btime pointInPolygonClassification(V, EV)   #123.196 μs

  123.196 ns (1 allocation: 32 bytes)


(::var"#pointInPolygonClassification0#13"{Matrix{Float64}, Vector{Vector{Int64}}}) (generic function with 1 method)

## Versione ottimizzata di pointInPolygonClassification

In [32]:
function edgecode1(c_int) #c_edge == 1
    if c_int == 0 return "p_on"
    elseif c_int == 4 crossingTest(1,2,status, counter) end 
end 

edgecode1 (generic function with 1 method)

In [33]:
function edgecode2(c_int) #c_edge == 2
    if c_int == 0 return "p_on"
    elseif c_int == 4 crossingTest(2,1,status, counter) end 
end 

edgecode2 (generic function with 1 method)

In [34]:
function edgecode3(c_int) #c_edge == 3
    if c_int == 0 return "p_on"
    elseif c_int == 4 counter += 1 end 
end 

edgecode3 (generic function with 1 method)

In [35]:
function edgecode4(c_un) #c_edge == 4
    if c_un == 3 return "p_on" end 
end 

edgecode4 (generic function with 1 method)

In [36]:
function edgecode5(c1,c2) #c_edge == 5
    if (c1==0) | (c2==0) return "p_on"
    else crossingTest(1,2,status, counter) end 
end 

edgecode5 (generic function with 1 method)

In [37]:
function edgecode6(c1,c2) #c_edge == 6
    if ((c1==0) | (c2==0)) return "p_on"
    else crossingTest(2,1,status, counter) end 
end 

edgecode6 (generic function with 1 method)

In [38]:
function edgecode7(counter) #c_edge == 7
    counter += 1
end

edgecode7 (generic function with 1 method)

In [39]:
function edgecode8(c_un) #c_edge == 8 (c_un == 6 su slide)
    if (c_un == 8) return "p_on" end   
end

edgecode8 (generic function with 1 method)

In [40]:
function edgecode9_10(c1,c2) #c_edge == 9/10
    if ((c1 ==0) | (c2==0)) return "p_on" end
end

edgecode9_10 (generic function with 1 method)

In [41]:
function edgecode11() #c_edge == 11
end

edgecode11 (generic function with 1 method)

In [42]:
function edgecode12(c_un) #c_edge = 12
    if (c_un == 12 ) return "p_on" end    
end

edgecode12 (generic function with 1 method)

In [43]:
function edgecode13(c1,c2) #c_edge = 13
    if (( c1 ==4) | (c2 == 4))
        crossingTest(1,2,status, counter) end
end

edgecode13 (generic function with 1 method)

In [44]:
function edgecode14(c1,c2) #c_edge = 14
    if (( c1 ==4) | (c2 == 4))
        crossingTest(2,1,status, counter) end
end

edgecode14 (generic function with 1 method)

In [45]:
function edgecode15(x1,x2,y1,y2,x,y)
    x_int = ((y-y2)*(x1-x2)/(y1-y2))+x2
    if x_int > x counter+=1
    elseif (x_int == x) return "p_on" end
end

edgecode15 (generic function with 1 method)

## Versione pointInPolygonClassificationMOD con funzioni separate

In [46]:
function pointInPolygonClassificationMOD(V,EV) 
    function pointInPolygonClassification0(pnt)
        x,y = pnt
        xmin,xmax,ymin,ymax = x,x,y,y
        tilecode = setTile([ymax,ymin,xmax,xmin])
        count,status = 0,0

        for (k,edge) in enumerate(EV)
            p1,p2 = V[:,edge[1]],V[:,edge[2]]
            (x1,y1),(x2,y2) = p1,p2
            c1,c2 = tilecode(p1),tilecode(p2)
            c_edge, c_un, c_int = c1⊻c2, c1|c2, c1&c2

            if (c_edge == 0) & (c_un == 0) return "p_on"
            elseif (c_edge == 12) & (c_un == c_edge) return "p_on"
            elseif c_edge == 3
                edgecode3(c_int)
            elseif c_edge == 15
               edgecode15(x1,x2,y1,y2,x,y)
            elseif (c_edge == 13) & ((c1==4) | (c2==4))
                    crossingTest(1,2,status,count)
            elseif (c_edge == 14) & ((c1==4) | (c2==4))
                    crossingTest(2,1,status,count)
            elseif c_edge == 7 edgecode7(counter)
            elseif c_edge == 11 count = count
            elseif c_edge == 1
               edgecode1(c_int)
            elseif c_edge == 2
                edgecode2(c_int)
            elseif (c_edge == 4) & (c_un == c_edge) return "p_on"
            elseif (c_edge == 8) & (c_un == c_edge) return "p_on"
            elseif c_edge == 5
                edgecode5(c1,c2)
            elseif c_edge == 6
                edgecode6(c1,c2)
            elseif (c_edge == 9) & ((c1==0) | (c2==0)) return "p_on"
            elseif (c_edge == 10) & ((c1==0) | (c2==0)) return "p_on"
            end
        end
        if (round(count)%2)==1
        	return "p_in"
        else
        	return "p_out"
        end
    end
    return pointInPolygonClassification0
end

pointInPolygonClassificationMOD (generic function with 1 method)

In [48]:
@btime pointInPolygonClassificationMOD(V,EV)  #122,009 μs

  122.009 ns (1 allocation: 32 bytes)


(::var"#pointInPolygonClassification0#12"{Matrix{Float64}, Vector{Vector{Int64}}}) (generic function with 1 method)

### Benchmark della funzione iniziale e modificata

funzione iniziale:

In [54]:
@benchmark pointInPolygonClassification(V, EV)

BenchmarkTools.Trial: 10000 samples with 900 evaluations.
 Range (min … max):  122.780 ns …   9.248 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     127.001 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   179.570 ns ± 281.921 ns  ┊ GC (mean ± σ):  1.30% ± 2.35%

  █▃▃▂▂▂▁▁▁▁▁▁▁▂▂▂▁▁▁                                           ▁
  ██████████████████████▇▇▇▇▇▆▆▆▆▆▆▄▆▅▅▅▅▅▅▃▄▃▃▄▄▄▃▂▄▄▂▄▃▃▃▂▃▄▃ █
  123 ns        Histogram: log(frequency) by time        650 ns <

 Memory estimate: 32 bytes, allocs estimate: 1.

funzione modificata:

In [53]:
@benchmark pointInPolygonClassificationMOD(V, EV)

BenchmarkTools.Trial: 10000 samples with 896 evaluations.
 Range (min … max):  121.691 ns …  14.682 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     124.946 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   189.439 ns ± 324.326 ns  ┊ GC (mean ± σ):  1.33% ± 2.32%

  █▂▂▂▂▂▂▁▁▂▃▂▂▁▁                                               ▁
  ███████████████████▇▇▇▇▆▆▅▅▆▅▅▅▅▅▅▅▅▄▄▅▄▃▄▅▅▄▄▅▄▃▄▃▃▄▃▅▂▅▄▃▄▅ █
  122 ns        Histogram: log(frequency) by time        882 ns <

 Memory estimate: 32 bytes, allocs estimate: 1.